На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурирующими, выяснить какая архитектура больше подходит для задачи сентимент анализа на данных с вебинара

1. построить свёрточную архитектуру
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше


In [8]:
#!pip install stop_words
#!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 17.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=18315ab8311a1ff1d03160e31c832004c4ddd580959401a7e1cadb8219dc7e19
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [9]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_val = pd.read_csv("/content/val.csv")



In [10]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [11]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [22]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [13]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [14]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [15]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [23]:
model_conv = Sequential()

model_conv.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model_conv.add(Masking(mask_value=0.0))

model_conv.add(Conv1D(64, 3, activation='relu'))
model_conv.add(Flatten())
model_conv.add(Dense(64, activation='relu'))
model_conv.add(Dropout(0.5))
model_conv.add(Dense(1, activation='sigmoid'))

model_conv.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model_conv.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 64)            5824      
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                102464    
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,851,593
Trainable params: 7,851,593
No

In [25]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_conv.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
319/319 [==============================] - 42s 128ms/step - loss: 0.5595 - accuracy: 0.7003 - val_loss: 0.4903 - val_accuracy: 0.7553
Epoch 2/20
319/319 [==============================] - 38s 120ms/step - loss: 0.3064 - accuracy: 0.8753 - val_loss: 0.5452 - val_accuracy: 0.7460


In [26]:
score = model_conv.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 12ms/step - loss: 0.5748 - accuracy: 0.7385


Test score: 0.574787974357605
Test accuracy: 0.7384825348854065


In [27]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 96s 290ms/step - loss: 0.5546 - accuracy: 0.7087 - val_loss: 0.4939 - val_accuracy: 0.7534
Epoch 2/10
319/319 [==============================] - 93s 293ms/step - loss: 0.3301 - accuracy: 0.8632 - val_loss: 0.5277 - val_accuracy: 0.7493


In [28]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 54ms/step - loss: 0.5733 - accuracy: 0.7368


Test score: 0.5732624530792236
Test accuracy: 0.7368072867393494


In [32]:
model_RNN_CNN = Sequential()

model_RNN_CNN.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_RNN_CNN.add(Masking(mask_value=0.0))
model_RNN_CNN.add(LSTM(64, return_sequences=True, recurrent_dropout=0.2))
model_RNN_CNN.add(Conv1D(64, 3, activation='relu'))
model_RNN_CNN.add(Flatten())
model_RNN_CNN.add(Dense(64, activation='relu'))
model_RNN_CNN.add(Dropout(0.5))
model_RNN_CNN.add(Dense(1, activation='sigmoid'))

model_RNN_CNN.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model_RNN_CNN.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 157s 321ms/step - loss: 0.5569 - accuracy: 0.7040 - val_loss: 0.4877 - val_accuracy: 0.7573
Epoch 2/10
319/319 [==============================] - 103s 324ms/step - loss: 0.3175 - accuracy: 0.8692 - val_loss: 0.5451 - val_accuracy: 0.7460


In [33]:
score = model_RNN_CNN.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 62ms/step - loss: 0.5844 - accuracy: 0.7374


Test score: 0.584389328956604
Test accuracy: 0.7373804450035095


Различия крайне малы, но если использовать более глубокую и объемную сетку то будет видно что архитектура RNN_CNN будет давать более хорошие результаты.